In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.firefox.options import Options

from datetime import datetime, timedelta

import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import numpy as np

import requests
from bs4 import BeautifulSoup

#from dbconnect import insert_articles,connect_db,insert_similar_articles

#import dbconnect  # Import the module itself
import importlib

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN

import spacy
from keybert import KeyBERT
from collections import defaultdict
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering

import re
import ast

In [ ]:
import scrape
import pandas as pd
class DataCollector:
    def __init__(self):
        self.data_list = []

    def append_data(self, new_data):
        self.data_list.append(new_data)

    def get_dataframe(self):
        return pd.DataFrame(self.data_list)
    
collector = DataCollector()

In [ ]:
scrape.wapo(collector)
df = collector.get_dataframe()

In [ ]:
def cnn_reader(driver):
    body = ' '.join([p.text for p in driver.find_elements(By.XPATH, '//*[@data-component-name="paragraph"]') if p])
    return body

def fox_reader(driver):
    ctx = driver.find_element(By.CLASS_NAME,'article-body')
    body_eles = [p for p in ctx.find_elements(By.TAG_NAME,'p')]
    if body_eles[0].get_attribute('class') != "speakable":
        body_eles.pop(0)
    body = ' '.join([p.text for p in body_eles])
    return body

def ap_reader(driver):
    ctx = driver.find_element(By.TAG_NAME,'bsp-story-page')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body

def npr_reader(driver):
    ctx = driver.find_element(By.ID,'storytext')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body

def hufpo_reader(driver):
    ctx = driver.find_element(By.ID,'entry-body')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body


def cbs_reader(driver):
    ctx = driver.find_element(By.CLASS_NAME,'content__body')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body

In [ ]:
#   Get text from articles
def read_articles(url_list):
    options = Options()
    options.add_argument('-headless')
    driver = webdriver.Firefox(options=options)
    driver.fullscreen_window()

    source_map = {
        'CNN': cnn_reader,
        'FOX': fox_reader,
        'AP': ap_reader,
        'NPR': npr_reader,
        'HuffPost': hufpo_reader,
        'CBS': cbs_reader
    }

    for url,source in url_list:
        print("going ",url)
        driver.get(url)

        reader_function = source_map[source]
        body = reader_function(driver)

        print(body)
        print("~~~~~~")

    driver.quit()


test = [
    ['https://www.cnn.com/2025/02/09/business/trump-tariffs-steel-aluminum/index.html','CNN'],
    ['https://www.cbsnews.com/news/trump-to-announce-tariffs-aluminum-steel-imports-monday/','CBS'],
    ['https://www.foxnews.com/us/colombian-nationals-arrested-connection-dozens-home-burglaries-oregon','FOX'],
    ['https://apnews.com/article/eric-adams-indictment-109ef48bd49bc8adc1850709c99bf666','AP'],
    ['https://www.npr.org/2025/02/11/nx-s1-5292505/einstein-ring-euclid','NPR'],
    ['https://www.huffpost.com/entry/nancy-gertner-donald-trump-move_n_67ab19ace4b025b52f75e895','HuffPost']
        ]

    
read_articles(test)

In [ ]:
#   get_full_articles.py
#       Read full article text and insert to database.
#       DO NOT CALL 'run_all'. This runs for the entirety of the database.
#
#
#       Questions: 
#           Read every article, or every article with matches (simart_id's)?
#               Every article. In summarization, create 2 different approaches to 1 article vs matching articles.
#


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.firefox.options import Options


import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import requests
from bs4 import BeautifulSoup

import importlib
import re
import ast

import utils

class DataCollector:
    def __init__(self):
        self.data_list = []

    def append_data(self, new_data):
        self.data_list.append(new_data)

    def get_dataframe(self):
        return pd.DataFrame(self.data_list)

#   Send the text corresponding to the article_id to database.
def insert_articles(connection, df):
    cursor = connection.cursor()
    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO article_text (article_id,article_content)
            VALUES (%s, %s)
            ON CONFLICT (article_id) DO NOTHING
        """, (row['article_id'], row['article_content']))
    
    connection.commit()
    cursor.close()

def cnn_reader(driver):
    body = ' '.join([p.text for p in driver.find_elements(By.XPATH, '//*[@data-component-name="paragraph"]') if p])
    return body

def fox_reader(driver):
    ctx = driver.find_element(By.CLASS_NAME,'article-body')
    body_eles = [p for p in ctx.find_elements(By.TAG_NAME,'p')]
    if body_eles[0].get_attribute('class') != "speakable":
        body_eles.pop(0)
    body = ' '.join([p.text for p in body_eles])
    return body

def ap_reader(driver):
    try:
        driver.find_element(By.ID,'onesignal-slidedown-cancel-button').click()
        driver.find_element(By.CSS_SELECTOR,'.bcpNotificationBarClose.bcpNotificationBarCloseIcon.bcpNotificationBarCloseTopRight').click()
    except:
        pass
    try:
        ctx = driver.find_element(By.TAG_NAME,'bsp-story-page')
    except:
        ctx = driver.find_element(By.CSS_SELECTOR,'.Page-content')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body

def npr_reader(driver):
    ctx = driver.find_element(By.ID,'storytext')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body

def hufpo_reader(driver):
    ctx = driver.find_element(By.ID,'entry-body')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body


def cbs_reader(driver):
    ctx = driver.find_element(By.CLASS_NAME,'content__body')
    body = ' '.join([p.text for p in ctx.find_elements(By.TAG_NAME,'p')])
    return body

#   Get text from articles
def read_articles(url_list):
    options = Options()
    options.add_argument('-headless')
    driver = webdriver.Firefox(options=options)
    driver.fullscreen_window()

    source_map = {
        'CNN': cnn_reader,
        'FOX': fox_reader,
        'AP': ap_reader,
        'NPR': npr_reader,
        'HuffPost': hufpo_reader,
        'CBS': cbs_reader
    }
    ready_df = pd.DataFrame(columns=['article_id','article_content'])
    for url,article_id,source in url_list:
        #   Paywalled sources
        if source in ['NYT','WAPO']:
            body = np.NAN
            ready_df.loc[len(ready_df)] = [article_id,body]
            continue
        #print(source,url)
        driver.get(url)
        try:
            reader_function = source_map[source]
            body = reader_function(driver)
        except:
            body = np.NAN

        ready_df.loc[len(ready_df)] = [article_id,body]

    driver.quit()
    return ready_df

#   DO NOT CALL.
#       Runs on all articles in database.
#       For more selective calls, get urls and send to read_articles
def run_all():
    connection = utils.connect_db()

    df = pd.read_sql_query("""
        SELECT a.article_id,a.url,a.article_source FROM articles a
        JOIN junct_simart_articles jsa ON jsa.article_id = a.article_id
        JOIN similar_articles sa ON jsa.simart_id = sa.simart_id
    """, con=connection)
    connection.close()

    print(df)
    url_list = df[['url','article_source']].values.tolist()
    res = read_articles(url_list)
    
def run_daily():
    connection = utils.connect_db()
    df = pd.read_sql_query("""
    SELECT 
	sa.simart_id,
	a.article_id,
	a.url,
	a.date,
	a.article_source
	FROM similar_articles sa
	JOIN junct_simart_articles jsa ON jsa.simart_id = sa.simart_id
	JOIN articles a ON a.article_id = jsa.article_id
	JOIN junct_simart_keywords jsk ON jsk.simart_id = sa.simart_id
	JOIN keywords kw ON kw.keyword_id = jsk.keyword_id
	WHERE sa.similar_weight >= 0.8
	AND EXISTS (
	SELECT 1
	FROM articles a2 
	JOIN junct_simart_articles jsa2 ON jsa2.article_id = a2.article_id
	WHERE jsa2.simart_id = sa.simart_id
	AND a2.date >= NOW() - INTERVAL '2 days'
	)
	GROUP BY sa.simart_id, sa.similar_weight, a.article_id, a.title, a.url, a.date, a.article_section, a.section_url, a.article_source, a.image, a.subheading
	ORDER BY sa.simart_id;
                           """,con=connection)

    url_list = df[['url','article_id','article_source']].values.tolist()
    res = read_articles(url_list)
    res['article_id'] = res['article_id'].astype(int)

    insert_articles(connection,res)
    connection.close()


run_daily()

In [187]:
#   Multi-doc Summarizer
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import importlib
import re
import ast

import utils

from nltk.corpus import stopwords

connection = utils.connect_db()
df = pd.read_sql_query("""
    SELECT 
        sa.simart_id,
        a.article_id,
        atx.article_content
	FROM similar_articles sa
        JOIN junct_simart_articles jsa ON jsa.simart_id = sa.simart_id
        JOIN articles a ON a.article_id = jsa.article_id
        JOIN article_text atx ON atx.article_id = a.article_id
	WHERE sa.similar_weight >= 0.8
	AND EXISTS (
        SELECT 1
        FROM articles a2 
        JOIN junct_simart_articles jsa2 ON jsa2.article_id = a2.article_id
        WHERE jsa2.simart_id = sa.simart_id
        AND a2.date >= NOW() - INTERVAL '2 days'
	)
	GROUP BY sa.simart_id,a.article_id,atx.article_content
	ORDER BY sa.simart_id;
                       """,con=connection)
connection.close()

In [188]:
grouped_df = df.groupby('simart_id')['article_content'].apply(list).reset_index()
grouped_df

,simart_id,article_content


In [ ]:
def preprocess(texts):
    cleaned = []
    for t in texts:
        #   Remove missing articles
        if pd.isna(t):
            continue
        t_clean = re.sub(r'[^\w\s\'$£€;\-:.]', '', str(t)) # remove excessive punctuation, keep periods
        t_clean = t_clean.replace('  ',' ')
        #   Empty texts
        if t_clean.lower() == 'nan' or not t_clean.strip():
            continue
        
        cleaned.append(t_clean)
        #   check for dupe text
    
    return cleaned

In [ ]:
def clean(df):
    rows_to_drop = []   # Drop empty matches after cleaning

    for i,r in df.iterrows():
        text_list = r['article_content']
        
        #   Clean texts
        cleaned_texts = preprocess(text_list)
        seen = set()
        unique_lst = [
            x
            for x in cleaned_texts 
            if not (x.strip().lower().replace("'", "").replace('"', "").replace("’", "") in seen or seen.add(x.strip().lower().replace("'", "").replace('"', "").replace("’", "")))
        ]
 
        df.at[i,'article_content'] = unique_lst
        if len(cleaned_texts) < 2:
            rows_to_drop.append(i)

    df.drop(rows_to_drop,inplace=True)
        
    return df

grouped_df = clean(grouped_df)

# Approach 1: TF-IDF vectorization
- Source: https://aclanthology.org/W00-0403.pdf \
    This study dives into an approach to multi-doc summarization for news articles specifically. \
    It highlights a centroid-based approach to collecting the most important sentences across all articles.




In [ ]:
from rapidfuzz import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

def fit_vectorizer():
    # Grab all
    connection = utils.connect_db()
    df = pd.read_sql_query("""
        SELECT jsa.simart_id, ax.article_id, ax.article_content 
        FROM junct_simart_articles jsa
        JOIN article_text ax ON jsa.article_id = ax.article_id
    """, con=connection)
    connection.close()

    grouped_df = df.groupby('simart_id')['article_content'].apply(list).reset_index()
    grouped_df = clean(grouped_df)  # Clean text

    # Function to remove highly similar duplicates
    def remove_dupes(df):
        new_df = df.copy()
        for i, row in new_df.iterrows():
            txt = row['article_content']
            to_remove = set()

            for x in txt:
                for y in txt:
                    if x == y:
                        continue
                    sim = fuzz.ratio(x, y)
                    if sim > 85:  # Near-duplicate detected
                        shorter, longer = (x, y) if len(x) < len(y) else (y, x)
                        to_remove.add(shorter)  # Keep the longer version

            # Remove duplicates for this row
            new_df.at[i, 'article_content'] = [t for t in txt if t not in to_remove]

        return new_df

    grouped_df = remove_dupes(grouped_df)

    # Check for remaining duplicates
    rows_to_drop = grouped_df[grouped_df['article_content'].apply(len) < 2].index
    grouped_df.drop(index=rows_to_drop, inplace=True)

    print(grouped_df)

    article_texts = []
    vectorizer = TfidfVectorizer(
        stop_words='english',
        token_pattern=r'(?u)\b[\w-]+\b'
    )

    for content_list in grouped_df['article_content']:
        for article in content_list:
            article_texts.append(article)


    vectorizer.fit(article_texts)
    #   Run once, save vectorizer
    joblib.dump(vectorizer,'tfidf_vectorizer.joblib')

fit_vectorizer()



In [ ]:
vectorizer = joblib.load('tfidf_vectorizer.joblib')

#### Calculate cluster centroid
The centroid, as seen in the study, is the average TF-IDF vector of all articles within a cluster of articles on the same topic. \
Thus, $
\text{Centroid} = \frac{\sum_{i=1}^{n} \mathbf{v}_i}{n}
$ For $\mathbf{v}_i$ = TF-IDF for each 1,..,n articles within a group

In [ ]:
for i,r in grouped_df.iterrows():
    group = r['article_content']
    print(group)

In [ ]:
grouped_df